# Database Implementation

In [46]:
# == MAGIC COMMANDS ====================================================================================================
# Enable matplotlib to display graphs directly in the notebook.
%matplotlib inline

# Load the autoreload extension to automatically reload external Python modules
%load_ext autoreload
%autoreload 2

# Set the precision of floating point numbers displayed in output for better readability
%precision 4


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'%.4f'

In [47]:
# ---- Standard Library Imports ----
import warnings
import sys
from pathlib import Path
import sqlite3

# ---- Third-Party Library Imports ----
from loguru import logger
import pandas as pd

# ---- Project-Specific Imports ----
from prod.utils import load_csv, check_full_system_environment
from prod.paths import DATABASE_DIR, RAW_DATA_DIR, PROCESSED_DATA_DIR, API_STATIC_DIR


# Filters
warnings.simplefilter(action='ignore', category=FutureWarning)

In [17]:
check_full_system_environment()

2025-01-30 13:57:46.283 | INFO     | prod.utils:check_full_system_environment:72 - Starting comprehensive system and GPU environment checks...
2025-01-30 13:57:46.283 | DEBUG    | prod.utils:check_full_system_environment:82 - Python Version: 3.12.4
2025-01-30 13:57:46.283 | DEBUG    | prod.utils:check_full_system_environment:83 - Operating System: Windows 10.0.22621
2025-01-30 13:57:46.283 | DEBUG    | prod.utils:check_full_system_environment:84 - Architecture: AMD64
2025-01-30 13:57:46.283 | DEBUG    | prod.utils:check_full_system_environment:85 - Number of Processors: 16

2025-01-30 13:57:46.298 | INFO     | prod.utils:check_full_system_environment:92 - CUDA nvcc output:
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Mon_Oct_24_19:40:05_Pacific_Daylight_Time_2022
Cuda compilation tools, release 12.0, V12.0.76
Build cuda_12.0.r12.0/compiler.31968024_0

2025-01-30 13:57:46.346 | INFO     | prod.utils:check_full_system_environment:103 - GPU com

## Setting up the database

In [48]:
# Ensure the database directory exists
DATABASE_DIR.mkdir(parents=True, exist_ok=True)

# Define the database file path
DB_PATH: Path = DATABASE_DIR / "credit_scoring.sqlite"

# Create the database (or connect if it exists)
conn: sqlite3.Connection = sqlite3.connect(DB_PATH)
conn.close()

## Exporting the .csv to the .db

### The raw customer data

In [49]:
# Dataset that will be sent to the database
df_raw: pd.DataFrame = load_csv(file_name = "application_test.csv", parent_path = RAW_DATA_DIR)

# Connect to the database
conn: sqlite3.Connection = sqlite3.connect(DB_PATH)

# Store dataset in SQLite as a table
df_raw.to_sql(name="customer_data", con=conn, if_exists="replace", index = False)

# Close the database connection
conn.close()

2025-01-30 14:52:41.038 | INFO     | prod.utils:load_csv:206 - Loaded     application_test.csv                     Shape:     (48744, 121)         Encoding:  ascii


### The data used by our model

In [50]:
# Dataset that will be sent to the database
df_predict: pd.DataFrame = load_csv(file_name = "04_prediction_df.csv", parent_path = PROCESSED_DATA_DIR)

# Connect to the database
conn: sqlite3.Connection = sqlite3.connect(DB_PATH)

# Store dataset in SQLite as a table
df_predict.to_sql(name="model_input_data", con=conn, if_exists="replace", index = False)

# Close the database connection
conn.close()

2025-01-30 14:52:45.433 | INFO     | prod.utils:load_csv:206 - Loaded     04_prediction_df.csv                     Shape:     (48744, 51)          Encoding:  ascii


### Descriptions about the features used by the model

In [51]:
# Dataset that will be sent to the database
df_description: pd.DataFrame = load_csv(file_name = "prediction_df_description.csv", parent_path = PROCESSED_DATA_DIR)

# Connect to the database
conn: sqlite3.Connection = sqlite3.connect(DB_PATH)

# Store dataset in SQLite as a table
df_description.to_sql(name="model_input_metadata", con=conn, if_exists="replace", index = False)

# Close the database connection
conn.close()

2025-01-30 14:52:53.402 | INFO     | prod.utils:load_csv:206 - Loaded     prediction_df_description.csv            Shape:     (51, 2)              Encoding:  ascii


## Database Scripts

In [52]:
# ====================================== GET THE LIST OF TABLES IN THE DATABASE ====================================== #
# Connect to the database
conn: sqlite3.Connection = sqlite3.connect(DB_PATH)
cursor: sqlite3.Cursor = conn.cursor()

# Fetch all table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Close the connection
conn.close()

# Log the tables found
logger.info(f"Tables in database: {[table[0] for table in tables]}")

2025-01-30 14:52:57.337 | INFO     | __main__:<module>:14 - Tables in database: ['customer_data', 'model_input_data', 'model_input_metadata']


In [53]:
# ======================================== GET THE COLUMNS OF A SPECIFIC TABLE ======================================= #
# Define the table name to inspect
table_name = "customer_data"

# Connect to the database
conn: sqlite3.Connection = sqlite3.connect(DB_PATH)
cursor: sqlite3.Cursor = conn.cursor()

# Fetch all column names from the specified table
cursor.execute(f"PRAGMA table_info({table_name});")
columns = cursor.fetchall()

# Close the connection
conn.close()

# Log the column names
logger.info(f"Columns in '{table_name}': {[col[1] for col in columns]}")


2025-01-30 15:23:13.507 | INFO     | __main__:<module>:17 - Columns in 'customer_data': ['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OWN_CAR_AGE', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START', 'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3', 'APARTMENTS_AVG', 'BASEMENTAREA_

## Extracting SHAP Explainer object 

In [54]:
import joblib
from prod.paths import API_DIR

# ==================================================================================================================== #
#                                                     CONFIGURATION                                                    #
# ==================================================================================================================== #

# Model Details
MODEL_NAME = "2025-01-17 - LGBMClassifier - business.joblib"
MODEL_PATH = API_DIR / "models" / MODEL_NAME
THRESHOLD = 0.48

# Scaler Details
SCALER_NAME = "2025-01-17 - RobustScaler.joblib"
SCALER_PATH = API_DIR / "models" / SCALER_NAME

# ==================================================================================================================== #
#                                            LOADING MODELS LOCALLY                                                    #
# ==================================================================================================================== #

# Load Model
try:
    model = joblib.load(MODEL_PATH)
    logger.success(f"Model loaded successfully from {MODEL_PATH}")
except Exception as e:
    logger.error(f"Error loading model: {e}")
    raise RuntimeError("An error occurred while loading the model. Please check the logs.")

# Load Scaler
try:
    robust_scaler = joblib.load(SCALER_PATH)
    logger.success(f"Scaler loaded successfully from {SCALER_PATH}")
except Exception as e:
    logger.error(f"Error loading scaler: {e}")
    raise RuntimeError("An error occurred while loading the scaler. Please check the logs.")

# ==================================================================================================================== #
#                                                     PREPROCESSING                                                    #
# ==================================================================================================================== #

# Apply RobustScaler with PassThrough
try:
    # Separate numeric and passthrough features
    numeric_features = [col for col in df_predict.select_dtypes(include=["number"]).columns if col != "SK_ID_CURR"]
    passthrough_features = ["SK_ID_CURR"]  # Explicit passthrough feature

    # Scale only numeric features using the loaded scaler
    numeric_scaled = robust_scaler.transform(df_predict[numeric_features])

    # Create a DataFrame for scaled numeric columns
    numeric_scaled_df = pd.DataFrame(numeric_scaled, columns=numeric_features, index=df_predict.index)

    # Combine scaled numeric features with passthrough columns
    passthrough_data = df_predict[passthrough_features]        # Keep passthrough columns untouched
    dataset_scaled = pd.concat([numeric_scaled_df, passthrough_data], axis=1)
    dataset_scaled = dataset_scaled[df_predict.columns]        # Reorder to match the original structure

    logger.success("Applied RobustScaler to numeric features successfully.")
except Exception as e:
    logger.error(f"Error applying RobustScaler: {e}")
    raise RuntimeError("An error occurred while scaling the df_predict. Please check the logs for more details.")

2025-01-30 19:47:40.004 | SUCCESS  | __main__:<module>:25 - Model loaded successfully from C:\Users\KDTB0620\Documents\Study\Open Classrooms\Git Repository\projet7\api\models\2025-01-17 - LGBMClassifier - business.joblib
2025-01-30 19:47:40.023 | SUCCESS  | __main__:<module>:33 - Scaler loaded successfully from C:\Users\KDTB0620\Documents\Study\Open Classrooms\Git Repository\projet7\api\models\2025-01-17 - RobustScaler.joblib
2025-01-30 19:47:40.128 | SUCCESS  | __main__:<module>:59 - Applied RobustScaler to numeric features successfully.


In [55]:
import shap

# ==================================================================================================================== #
#                                          CREATE SHAP EXPLAINER                                                        #
# ==================================================================================================================== #

try:
    logger.info("Computing SHAP Explainer...")

    # Ensure we exclude 'SK_ID_CURR' since it's not a feature
    features_only = dataset_scaled.drop(columns=["SK_ID_CURR"], errors="ignore")

    # Create a SHAP explainer using the trained model
    explainer = shap.Explainer(model, features_only)

    logger.success("SHAP Explainer created successfully.")

except Exception as e:
    logger.error(f"Error computing SHAP Explainer: {e}")
    raise RuntimeError("An error occurred while computing the SHAP explainer.")


2025-01-30 19:53:59.855 | INFO     | __main__:<module>:8 - Computing SHAP Explainer...
2025-01-30 19:54:00.362 | SUCCESS  | __main__:<module>:16 - SHAP Explainer created successfully.


In [56]:
# Define the save path
MODEL_PATH = API_DIR / "models" / "shap_explainer.joblib"

try:
    logger.info(f"Saving SHAP Explainer to {MODEL_PATH}...")

    # Save the SHAP explainer
    joblib.dump(explainer, MODEL_PATH)

    logger.success(f"SHAP Explainer saved successfully at {MODEL_PATH}")

except Exception as e:
    logger.error(f"Error saving SHAP Explainer: {e}")
    raise RuntimeError("An error occurred while saving the SHAP explainer.")

2025-01-30 20:27:39.325 | INFO     | __main__:<module>:5 - Saving SHAP Explainer to C:\Users\KDTB0620\Documents\Study\Open Classrooms\Git Repository\projet7\api\models\shap_explainer.joblib...
2025-01-30 20:27:39.435 | SUCCESS  | __main__:<module>:10 - SHAP Explainer saved successfully at C:\Users\KDTB0620\Documents\Study\Open Classrooms\Git Repository\projet7\api\models\shap_explainer.joblib


In [ ]:
# ==================================================================================================================== #
#                                       PRECOMPUTE SHAP GLOBAL FEATURE IMPORTANCE                                      #
# ==================================================================================================================== #
import matplotlib.pyplot as plt


# Config
OUTPUT_FILE = API_STATIC_DIR / "model_predictors.png"

try:
    if OUTPUT_FILE.exists():
        logger.info(f"The file {OUTPUT_FILE} already exists. Skipping SHAP computation.")
    else:
        # Take a sample of 48,000 rows from the scaled dataset
        sample_data = dataset_scaled.sample(n=48000, random_state=42)

        # Ensure that the sample excludes the target column if present
        features_only = sample_data.drop(columns=["SK_ID_CURR"], errors="ignore")

        # Create a SHAP explainer for the model
        explainer = shap.Explainer(model, features_only)

        # Calculate SHAP values for the sample data
        shap_values = explainer(features_only, check_additivity=False)

        # Generate a beeswarm plot for the top 15 features
        plt.figure(figsize=(13, 9))
        shap.summary_plot(
            shap_values=shap_values,
            features=features_only,
            plot_type="violin",
            max_display=15,
            show=False
            )

        # Add title with increased padding
        plt.title("Top 15 Model Predictors", pad=20, fontsize=16)
        plt.tight_layout()

        # Save the plot as a .png file in the specified location
        plt.savefig(OUTPUT_FILE)
        plt.close()  # Close the plot to free memory
        logger.success(f"SHAP beeswarm plot saved successfully at {OUTPUT_FILE}.")
except Exception as e:
    logger.error(f"Error computing SHAP feature impact: {e}")
    raise RuntimeError("An error occurred while precomputing SHAP feature impact. Please check the logs.")
